In [ ]:
from shared.preface import *
import shared.functions as fct
import shared.my_units as my
import shared.control_center as CC

# Calculating & Investigating final number density plot

In [ ]:
from shared.preface import *
import shared.functions as fct
import shared.my_units as my
import shared.control_center as CC

# Load data, which function uses.
u_all = fct.load_u_sim()
def number_density_1_mass(m_nu_eV, average=False):

    n_nus = np.zeros(len(m_nu_eV))
    for i, m_eV in enumerate(m_nu_eV):

        # Get momenta.
        p, _ = fct.u_to_p_eV(u_all, m_eV)

        if average and m_eV.value >= 0.1:
            # Calculate number density, values are averaged for z in [3.5, 4].
            idx = np.array(np.where(CC.ZEDS >= 3.5)).flatten()

            temp = np.zeros(len(idx))
            for j,k in enumerate(idx):
                val = fct.number_density(p[:,0], p[:,k]).value
                temp[j] = val

            n_nus[i] = np.mean(temp)

        else:
            n_nus[i] = fct.number_density(p[:,0], p[:,-1]).value

    np.save('neutrino_data/number_densities.npy', n_nus)


# 10 to 300 meV like in the paper
mass_range_eV = np.geomspace(0.01, 0.3, 100)*unit.eV

number_density_1_mass(mass_range_eV, average=True)
n_nus = np.load('neutrino_data/number_densities.npy')/CC.N0

# Plotting:
fig, ax = plt.subplots(1,1, figsize=(8,8))

ax.plot(mass_range_eV*1e3, (n_nus-1))
x_ends = [1e1, 3*1e2]
y_ends = [3*1e-3, 4]
ax.scatter(x_ends, y_ends, s=12, marker='x', c='r')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title(f'NFW only - {CC.NR_OF_NEUTRINOS} neutrinos')
ax.set_xlabel(r'$m_{\nu}$ [meV]')
ax.set_ylabel(r'$n_{\nu} / n_{\nu, 0}$')
ax.set_ylim(1e-3, 1e1)
plt.grid(True, which="both", ls="-")

ax.yaxis.set_major_formatter(ticker.FuncFormatter(fct.y_fmt))

plt.savefig(f'check_plots/densities_{CC.NR_OF_NEUTRINOS}_nus.pdf')
plt.show()

print('Max value:', np.max(n_nus), np.max(n_nus/CC.N0))

## Fermi-Dirac shape

In [ ]:
# Load velocities of sim.
u_all = fct.load_u_sim()
u0, u1 = u_all[:,0,:], u_all[:,-1,:]

# Position of earth w.r.t Milky Way NFW halo center.
x1, x2, x3 = 8.5, 0., 0.
x0 = np.array([x1, x2, x3])

fig, axs = plt.subplots(2,2, figsize=(12,10))
fig.suptitle(
    'Mometum Phase-Space distributions' 
    '\n'
    f'({CC.NR_OF_NEUTRINOS} neutrinos)',
    fontsize=18)

masses = np.array([0.01, 0.05, 0.1, 0.3])*unit.eV
for j, m in enumerate(masses):

    k = j

    # Get momenta.
    p0, y0 = fct.u_to_p_eV(u0, m)
    p1, y1 = fct.u_to_p_eV(u1, m)

    # Sort.
    order = p0.argsort()
    p0_sort, p1_sort = p0[order], p1[order]
    y0_sort, y1_sort = y0[order], y1[order]

    i = 0
    if j in (2,3):
        i = 1
        j -= 2
    
    # Escape momentum.
    p_esc_eV, y_esc = fct.escape_momentum(x0, 0., my.rho0_NFW, my.Mvir_NFW, m)
    axs[i,j].axvline(y_esc, c='k', ls='-.', label='y_esc')

    # Original Fermi-Dirac distr.
    pOG = np.geomspace(CC.LOWER*my.T_nu_eV, CC.UPPER*my.T_nu_eV, len(p0_sort))
    FDvalsOG = fct.Fermi_Dirac(pOG)
    yOG = (pOG/my.T_nu_eV).value

    # Average values below escape momentum.
    indices = np.argwhere(y1_sort <= y_esc)
    p1_below_esc = p1_sort[indices]
    # p1_sort[indices] = p_esc_eV


    # Fermi-Dirac distr. of initial momentum (by using p1 momenta at z_back)
    FDvals = fct.Fermi_Dirac(p1_sort)
    # FDvals[indices] = np.mean(FDvals[indices])
    # print(FDvals[:8])

    print('Max FDval from sim:', np.max(FDvals))



    axs[i,j].loglog(y0_sort, FDvals, label='Simulation', c='red', alpha=0.6)
    axs[i,j].loglog(yOG, FDvalsOG, label='Fermi-Dirac', c='blue')
    axs[i,j].set_title(f'{m.value} eV')
    axs[i,j].set_ylabel('FD(p)')
    axs[i,j].set_xlabel(r'$y = p / T_{\nu,0}$')
    axs[i,j].legend(loc='lower left')
    axs[i,j].set_ylim(1e-5, 1e0)


    # if i==0 and j==0:
    #     axs[i,j].set_xlim(4*1e-2,3*1e-1)
    #     axs[i,j].set_ylim(1e-1,5*np.max(y0_sort))

    # if i==1 and j==0:
    #     axs[i,j].set_xlim(4*1e-1,8*1e0)
    #     axs[i,j].set_ylim(1e-4,5*np.max(y0_sort))

    # if i==1 and j==1:
    #     axs[i,j].set_xlim(1e0,5*1e1)
    #     axs[i,j].set_ylim(1e-9,5*np.max(y0_sort))


plt.savefig('check_plots/fermi_dirac_distr.pdf')
plt.show()

## Initial and final momenta.

In [ ]:
# Load velocities of sim.
u_all = fct.load_u_sim()
u0, u1 = u_all[:,0,:], u_all[:,-1,:]

# Position of earth w.r.t Milky Way NFW halo center.
x1, x2, x3 = 8.5, 0., 0.
x0 = np.array([x1, x2, x3])

fig, axs = plt.subplots(2,2, figsize=(12,10))
fig.suptitle(
    'Initial and final momenta (sorted)' 
    '\n'
    f'({CC.NR_OF_NEUTRINOS} neutrinos)',
    fontsize=18)

masses = np.array([0.01, 0.05, 0.1, 0.3])*unit.eV
for j, m in enumerate(masses):

    # Get momenta.
    p0, y0 = fct.u_to_p_eV(u0, m)
    p1, y1 = fct.u_to_p_eV(u1, m)

    # Sort.
    order = p0.argsort()
    p0_sort, p1_sort = p0[order], p1[order]
    y0_sort, y1_sort = y0[order], y1[order]

    i = 0
    if j in (2,3):
        i = 1
        j -= 2
    
    # Escape momentum.
    p_esc_eV, y_esc = fct.escape_momentum(x0, 0., my.rho0_NFW, my.Mvir_NFW, m)
    axs[i,j].axhline(y_esc, c='k', ls='-.', label='y_esc')

    # Average momenta below escape momentum.

    x = np.arange(CC.NR_OF_NEUTRINOS)
    axs[i,j].semilogy(x, y0_sort, label='initial momentum')
    axs[i,j].semilogy(x, y1_sort, label='final momentum', alpha=0.6)
    axs[i,j].set_title(f'{m.value} eV')
    axs[i,j].set_xlabel('Neutrino number')
    axs[i,j].set_ylabel(r'$p / T_{\nu,0}$')
    axs[i,j].legend(loc='lower right')

plt.show()